## Explore the Data Observatory catalog

The Data Observatory is a a spatial data platform that enables Data Scientists to augment their data and broaden their analysis. It offers a wide range of datasets from around the globe in a spatial data repository.

This guide is intended for those who are going to start augmenting their own data using CARTOframes and are willing to explore our public Data Observatory catalog on the seek of the datasets that best fit their use cases and analyses.

**Note: The catalog is public and you don't need a CARTO account to search for available datasets**

### Looking for demographics and financial data in the US in the catalog

In this guide we are going to filter the Data Observatory catalog looking for demographics and financial data in the US.

The catalog is comprised of thousands of curated spatial datasets, so when searching for
data the easiest way to find out what you are looking for is make use of a feceted search. A faceted (or hierarchical) search allows you to narrow down search results by applying multiple filters based on faceted classification of the catalog datasets.

Datasets are organized in three main hirearchies:

- Country
- Category
- Geography (or spatial resolution)

For our analysis we are looking for demographics and financial datasets in the US with a spatial resolution at the level of block groups. 

First we can start for discovering which available geographies (orspatial resolutions) we have for demographics data in the US, by filtering the `catalog` by `country` and `category` and listing the available `geographies`.

Let's start exploring the available categories of data for the US:

In [ ]:
from cartoframes.data.observatory import Catalog
Catalog().country('usa').categories

For the case of the US, the Data Observatory provides six different categories of datasets. Let's discover the available spatial resolutions for the demographics category (which at a first sight will contain the population data we need).

In [ ]:
from cartoframes.data.observatory import Catalog
geographies = Catalog().country('usa').category('demographics').geographies
geographies

Let's filter the geographies by those that contain information at the level of blockgroup. For that purpose we are converting the geographies to a pandas `DataFrame` and search for the string `blockgroup` in the `id` of the geographies:

In [ ]:
df = geographies.to_dataframe()
df[df['id'].str.contains('blockgroup', case=False, na=False)]

We have three available datasets, from three different providers: Michael Bauer International, Open Data and AGS. For this example, we are going to look for demographic datasets for the AGS blockgroups geography `ags_blockgroup_1c63771c`:

In [ ]:
datasets = Catalog().country('usa').category('demographics').geography('ags_blockgroup_1c63771c').datasets
datasets

In [ ]:
datasets.to_dataframe()

They comprise different information: consumer spending, retail potential, consumer profiles, etc.

At a first sight, it looks the dataset with `data_source_id: sociodemographic` might contain the population information we are looking for. Let's try to understand a little bit better what data this dataset contains by looking at its variables:

In [ ]:
from cartoframes.data.observatory import Dataset
dataset = Dataset.get('ags_sociodemogr_e92b1637')
variables = dataset.variables
variables

In [ ]:
from cartoframes.data.observatory import Dataset
vdf = variables.to_dataframe()
vdf

We can see there are several variables related to population, so this is the `Dataset` we are looking for.

In [ ]:
vdf[vdf['description'].str.contains('pop', case=False, na=False)]

We can follow the very same process to discover `financial` datasets, let's see how it works by first listing the geographies available for the category `financial` in the US:

In [ ]:
Catalog().country('usa').category('financial').geographies

We can clearly identify a geography at the blockgroup resolution, provided by Mastercard:

In [ ]:
from cartoframes.data.observatory import Geography
Geography.get('mc_blockgroup_c4b8da4c').to_dict()

Now we can list the available datasets provided by Mastercard for the US Census blockgroups spatial resolution:

In [ ]:
Catalog().country('usa').category('financial').geography('mc_blockgroup_c4b8da4c').datasets.to_dataframe()

Let's finally inspect the variables available in the dataset:

In [ ]:
Dataset.get('mc_mrli_35402a9d').variables

### Dataset and variables metadata

The Data Observatory catalog is not only a repository of curated spatial datasets, it also contains valuable information that helps on understanding better the underlying data for every dataset, so you can take an informed decision on what data best fits your problem.

Some of the augmented metadata you can find for each dataset in the catalog is:

- `head` and `tail` methods to get a glimpse of the actual data. This helps you to understand the available columns, data types, etc. To start modelling your problem right away.
- `geom_coverage` to visualize on a map the geographical coverage of the data in the `Dataset`.
- `counts`, `fields_by_type` and a full `describe` method with stats of the actual values in the dataset, such as: average, stdev, quantiles, min, max, median for each of the variables of the dataset.

You don't need a subscription to a dataset to be able to query the augmented metadata, it's just publicly available for anyone exploring the Data Observatory catalog.

Let's overview some of that information, starting by getting a glimpse of the ten first or last rows of the actual data of the dataset:

In [ ]:
from cartoframes.data.observatory import Dataset
dataset = Dataset.get('ags_sociodemogr_e92b1637')

In [ ]:
dataset.head()

Alternatively, you can get the last ten ones with `dataset.tail()`

An overview of the coverage of the dataset

In [ ]:
dataset.geom_coverage()

Some stats about the dataset:

In [ ]:
dataset.counts()

In [ ]:
dataset.fields_by_type()

In [ ]:
dataset.describe()

Every `Dataset` instance in the catalog contains other useful metadata:

- slug: A short ID
- name and description: Free text attributes
- country
- geography: Every dataset is related to a Geography instance
- category
- provider
- data source
- lang
- temporal aggregation
- time coverage
- update frequency
- version
- is_public_data: whether you need a license to use the dataset for enrichment purposes or not

In [ ]:
dataset.to_dict()

There's also some intersting metadata, for each variable in the dataset:

- id
- slug: A short ID
- name and description
- column_name: Actual column name in the table that contains the data
- db_type: SQL type in the database
- dataset_id
- agg_method: Aggregation method used
- temporal aggregation and time coverage

Variables are the most important asset in the catalog and when exploring datasets in the Data Observatory catalog it's very important that you understand clearly what variables are available to enrich your own data.

For each `Variable` in each dataset, the Data Observatory provides (as it does with datasets) a set of methods and attributes to understand their underlaying data.

Some of them are:

- `head` and `tail` methods to get a glimpse of the actual data and start modelling your problem right away.
- `counts`, `quantiles` and a full `describe` method with stats of the actual values in the dataset, such as: average, stdev, quantiles, min, max, median for each of the variables of the dataset.
- an `histogram` plot with the distribution of the values on each variable.

Let's overview some of that augmented metadata for the variables in the AGS population dataset.

In [ ]:
from cartoframes.data.observatory import Variable
variable = Variable.get('POPPY_946f4ed6')
variable

In [ ]:
variable.to_dict()

There's also some utility methods ot understand the underlying data for each variable:

In [ ]:
variable.head()

In [ ]:
variable.counts()

In [ ]:
variable.quantiles()

In [ ]:
variable.histogram()

In [ ]:
variable.describe()

### Subscribe to a Dataset in the catalog

Once you have explored the catalog and have detected a dataset with the variables you need for your analysis and the right spatial resolution, you have to look at the `is_public_data` to know if you can just use it from CARTOframes or you first need to subscribe for a license.

Subscriptions to datasets allow you to use them from CARTOframes to enrich your own data or to download them. See the enrichment guides for more information about this.

Let's see the dataset and geography in our previous example:

In [ ]:
dataset = Dataset.get('ags_sociodemogr_e92b1637')

In [ ]:
dataset.is_public_data

In [ ]:
from cartoframes.data.observatory import Geography
geography = Geography.get(dataset.geography)

In [ ]:
geography.is_public_data

Both `dataset` and `geography` are not public data, that means you need a subscription to be able to use them to enrich your own data.

**To subscribe to data in the Data Observatory catalog you need a CARTO account with access to Data Observatory**

In [ ]:
from cartoframes.auth import set_default_credentials
YOUR_CARTO_USER_NAME = ''
YOUR_CARTO_API_KEY = ''
set_default_credentials(username=YOUR_CARTO_USER_NAME, api_key=YOUR_CARTO_API_KEY)
dataset.subscribe()

In [ ]:
geography.subscribe()

**Licenses to data in the Data Observatory grant you the right to use the data subscribed for the period of one year. Every dataset or geography you want to use to enrich your own data, as lons as they are not public data, require a valid license.**

You can check the actual status of your subscriptions directly from the catalog.

In [ ]:
Catalog().subscriptions()

## About nested filters in the Catalog instance

**Note that every time you search the catalog you create a new instance of the `Catalog` class. Alternatively, when applying `country`, `category` and `geography` filters a catalog instance, you can reuse the same instance of the `catalog` by using the `catalog.clean_filters()` method.**

So for example, if you've filtered the catalog this way:

In [ ]:
catalog = Catalog()
catalog.country('usa').category('demographics').datasets

And now you want to take the `financial` datasets for the use, you should:

1. Create a new instance of the catalog: `catalog = Catalog()`
2. Call to `catalog.clean_filters()` over the existing instance.

Another point to remark is that, altough a recommended way to discover data is nesting filters over a `Catalog` instance, you don't need to follow the complete hierarchy (`country`, `category`, `geography`) to list the available datasets.

Alternatively, you can just list all the datasets in the `US` or list all the datasets for the `demographics` category, and continue exploring the catalog locally with pandas.

Let's see an example of that, in which we filter public data for the `demographics` category world wide:

In [ ]:
df = Catalog().category('demographics').datasets.to_dataframe()
df[df['is_public_data'] == True]

### Learn more

We recommend you to check also these resources if you want to know more about the Data Observatory catalog:

- The CARTOframes Enrichment guides and examples
- [our public website](https://carto.com/platform/location-data-streams/)
- Your user dashboard: Under the data section
- The CARTOframes catalog reference 